In [1]:
import torch
import torchmetrics
from torch import nn
import module.resnet as resnet
import module.utils as utils
from torchinfo import summary
from torchvision import transforms
from module.data import CfImageDataset
from tqdm import tqdm
import time
import numpy as np
import pandas as pd

In [2]:
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.benchmark = True

In [3]:
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip()
])

In [4]:
train_dataset = CfImageDataset("F:/DATA/classify-leaves/train.csv", "F:/DATA/classify-leaves/", mode="train", transform=data_transforms)
valid_dataset = CfImageDataset("F:/DATA/classify-leaves/train.csv", "F:/DATA/classify-leaves/", mode="valid")

In [6]:
train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=0,
        drop_last=True
    )
valid_loader = torch.utils.data.DataLoader(
        dataset=valid_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=0
    )

In [7]:
net = resnet.ResNet(64, 176, 4)

In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    net = net.cuda()

In [23]:
optim = torch.optim.Adam(net.parameters(), lr=0.001)
Accuracy = torchmetrics.Accuracy().to(torch.device("cuda", 0))
Mean = torchmetrics.MeanMetric().to(torch.device("cuda", 0))

In [35]:
def train_step(x, y):
    pred = net(x)
    loss = loss_fn(pred, y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    Accuracy.update(pred, y)
    Mean.update(loss)

In [36]:
def eval_step(x, y):
    pred = net(x)
    loss = loss_fn(pred, y)
    Accuracy.update(pred, y)
    Mean.update(loss)

In [39]:
n_iter = 0
for epoch in range(0, 10):
    # set models to train mode
    net.train()
    # use prefetch_generator and tqdm for iterating through data
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    start_time = time.time()

    # for loop going through dataset
    for i, data in pbar:
        # data preparation
        img, label = data
        if use_cuda:
            img = img.cuda()
            label = label.cuda()
        prepare_time = start_time - time.time()
        train_step(img, label)
        # compute computation time and *compute_efficiency*
        process_time = start_time - time.time() - prepare_time
        pbar.set_description("Compute efficiency: {:.2f}, epoch: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(
            process_time / (process_time + prepare_time), epoch, 10, Mean.compute(), Accuracy.compute()))
        start_time = time.time()
    Accuracy.reset()
    Mean.reset()
    if epoch % 2 == 2-1:
        # bring models to evaluation mode
        net.eval()
        #do some tests
        pbar = tqdm(enumerate(valid_loader),
                total=len(valid_loader))
        for i, data in pbar:
            img, label = data
            if use_cuda:
                img = img.cuda()
                label = label.cuda()
            eval_step(img, label)
        print("valid loss: {}, valid accuracy: {}".format(Mean.compute(), Accuracy.compute()))
        Accuracy.reset()
        Mean.reset()

Compute efficiency: 0.69, epoch: 0/10, loss: 0.45663, accuracy: 0.86831:   2%|▏         | 21/1032 [00:02<02:17,  7.35it/s]


KeyboardInterrupt: 